In [ ]:
import os, sys
from pathlib import Path
import cortex
import nibabel as nib
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as pl
import time
import platform

In [ ]:
if platform.node()=='theCryingMachine.local':
    deriv_dir = Path('/Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-001/derivatives/')
elif platform.node()=='Smoldering-Corpse-Bar.local':
    deriv_dir = Path('/Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-001/derivatives/')
elif platform.node()=='node230':
    raise ValueError('hmmm I dont think it works on a server. Your are working on {}'.format(platform.node()))
else:
    raise ValueError('No idea which computer this is. The computer name is {}'.format(platform.node()))

print('Running on {}'.format(platform.node()))
print('Current deriv folder is {}'.format(deriv_dir))

print('cortex.database.default_filestore: {}'.format(cortex.database.default_filestore))
print('cortex.options.usercfg: {}'.format(cortex.options.usercfg))

In [ ]:
data_dir = Path(deriv_dir, 'GLMsingle_test', 'GLMsingle')
fs_dir = Path(deriv_dir, 'freesurfer')
fs_subj_dir = Path(fs_dir, 'sub-001')

In [ ]:
static_imgs = False
web_view = True

# polar angle and prf mapping stat maps
for subj_nr in [1]:
    sub = "sub-%s" % str(subj_nr).zfill(3)
    xfmname = "fmriprep_fsT1"
    dd = os.path.join(data_dir, sub, "ses-1", "deriv")
    realf = os.path.join(dd, "polar_real_all.nii.gz")
    imagf = os.path.join(dd, "polar_imag_all.nii.gz")
    srsqf = os.path.join(dd, "all_rsq_s.nii.gz")
    try:
        os.makedirs(os.path.join(data_dir, sub, "ses-1", "figs", "surf"))
    except:
        pass
    # # # reading in the psc file data
    # print('loading psc dataset')
    # vpsc = cortex.Volume(psc_file, 'sub-%s' % str(subj_nr).zfill(3),
    #                      'all_final', cmap='BuBkRd', vmin=-0.5, vmax=0.5)
    # polar angle data
    print("loading complex dataset")
    reald, imagd = nb.load(realf).get_data(), nb.load(imagf).get_data()
    polcomp = reald + 1j * imagd
    angs = np.angle(polcomp)
    rsq = np.abs(polcomp)
    s = np.ones(angs.shape)
    v = np.ones(angs.shape)
    angs_n = (angs + np.pi) / (np.pi * 2.0)
    # make discrete angles for clarity
    angle_offset = 0.08
    angs_discrete = np.fmod(angle_offset + (np.floor(angs_n * 6) / 6.0), 1.0)
    # convert angles to colors, using correlations as weights
    hsv = np.zeros(list(angs.shape) + [3])
    hsv[..., 0] = angs_discrete  # angs_discrete  # angs_n
    hsv[..., 1] = np.sqrt(rsq)  # np.ones_like(rsq)  # np.sqrt(rsq)
    # np.nan_to_num(rsq ** -3) # np.ones_like(rsq)#n
    hsv[..., 2] = np.sqrt(rsq)  # np.ones_like(rsq)
    alpha_mask = (rsq <= 0.2).T
    alpha = np.sqrt(rsq).T * 5
    alpha[alpha_mask] = 0
    alpha = np.ones(alpha.shape)
    rgb = colors.hsv_to_rgb(hsv)
    ###########################################################################################
    # Datasets
    ###########################################################################################
    # r-squared data
    print("loading rsq dataset")
    vsrsq = cortex.Volume(
        data=nb.load(srsqf).get_data().T,
        subject="sub-%s" % str(subj_nr).zfill(3),  # a.transpose((2,1,0))
        xfmname=xfmname,
        cmap="BuBkRd",
        vmin=-1,
        vmax=1,
        with_dropout=8,
    )
    # mask=yeo_7_dmn_filename)
    vrgba = cortex.VolumeRGB(
        red=rgb[..., 0].T,
        green=rgb[..., 1].T,
        blue=rgb[..., 2].T,
        subject="sub-%s" % str(subj_nr).zfill(3),
        # np.sqrt(rsq).T * 5, # np.ones(rsq.T.shape), # np.sqrt(rsq).T * 5, #
        alpha=alpha,
        xfmname=xfmname,
    )
    # mask=yeo_7_dmn_filename)
    # rework the data
    # normalize
    # vpsc.data = (vpsc.data - (vpsc.data.mean(axis=0)
    #                           [np.newaxis, ...])) / (vpsc.data.std(axis=0)[np.newaxis, ...])
    # # weight with signed rsq
    # vpsc.data *= np.sqrt(np.abs(vsrsq.data[np.newaxis, ...]))
    # creating a re-usable dataset
    ds = cortex.Dataset(rsq_s=vsrsq, polar=vrgba)
    ds.save(os.path.join(data_dir, sub, "ses-1", "h5", "cortex_pa-rsq.h5"))
    # print('created all volume datasets')
    # # static figures
    if static_imgs:
        cortex.quickshow(
            braindata=vsrsq,
            with_colorbar=False,
            depth=0.0,
            thick=1,
            height=2048,
            sampler="nearest",
            with_curvature=True,
            with_dropout=False,
            with_labels=True,
        )
        pl.gca().invert_yaxis()
        pl.savefig(
            os.path.join(data_dir, sub, "ses-1", "figs", "surf", "all_rsq_s.pdf")
        )
        cortex.quickshow(
            braindata=vrgba,
            with_colorbar=False,
            depth=0.0,
            thick=1,
            height=2048,
            sampler="nearest",
            with_curvature=True,
            with_dropout=False,
            with_labels=True,
        )
        pl.gca().invert_yaxis()
        pl.savefig(
            os.path.join(data_dir, sub, "ses-1", "figs", "surf", "all_polar.pdf")
        )
        pl.close("all")
    # cortex.utils.add_roi(data=vrgba, name='nprf_polar', add_path=False, open_inkscape=False)
    # cortex.utils.add_roi(data=vsrsq, name='nprf_rsq', add_path=False, open_inkscape=False)
    if web_view:
        print("creating web view")
        # webview for movies and comparisons
        for d, nm, depth in zip([vsrsq, vrgba], ["rsq", "polar"], [0, 0]):
            ds = cortex.Dataset(**{nm: d})
            handle = cortex.webgl.show(data=ds, recache=False, port=12001)
            file_pattern = "{base}_{view}_{nm}.png"
            time.sleep(20.0)
            # projection parameters
            basic = dict(
                radius=300, depth=depth, specularity=0, unfold=0.5, contrast=0
            )  # projection=['orthographic'],
            # different views available, more views can be added  and the
            # existing list can be removed
            views = dict(
                lateral=dict(altitude=90.5, azimuth=181, pivot=180),
                medial=dict(altitude=90.5, azimuth=0, pivot=180),
                front=dict(altitude=90.5, azimuth=0, pivot=0),
                back=dict(altitude=90.5, azimuth=181, pivot=0),
                top=dict(altitude=0, azimuth=180, pivot=0),
                bottom=dict(altitude=180, azimuth=0, pivot=0),
            )
            # utility functions to set the different views
            prefix = dict(
                altitude="camera.",
                azimuth="camera.",
                pivot="surface.{subject}.",
                radius="camera.",
                unfold="surface.{subject}.",
                depth="surface.{subject}.",
                specularity="surface.{subject}.",
                contrast="surface.curvature.",
            )
            _tolists = lambda p: {prefix[k] + k: [v] for k, v in p.items()}
            _combine = lambda a, b: (lambda c: [c, c.update(b)][0])(dict(a))
            # Save images by iterating over the different views and surfaces
            for view, vparams in views.items():
                # Combine basic, view, and surface parameters
                params = _combine(basic, vparams)
                # Set the view
                handle._set_view(**_tolists(params))
                # Save image
                filename = file_pattern.format(base=sub, view=view, nm=nm)
                output_path = os.path.join(
                    data_dir, sub, "ses-1", "figs", "surf", filename
                )
                handle.getImage(output_path, size=(3840, 2160))
                # the block below trims the edges of the image:
                # wait for image to be written
                while not os.path.exists(output_path):
                    pass
                time.sleep(1.5)
                # try:
                #     import subprocess
                #     subprocess.call(['convert', '-trim', output_path, output_path])
                # except:
                #     pass
            # Close the window!
            handle.close()